In [1]:
# Gerekli Kütüphaneleri Yükleyin
import pandas as pd
import psutil
import subprocess
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# GPU Kullanımını Ölçmek İçin Fonksiyon
def get_gpu_usage():
    try:
        result = subprocess.check_output(["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,noheader,nounits"])
        gpu_usage = float(result.decode("utf-8").strip())
    except:
        gpu_usage = 0.0  # GPU yoksa veya erişilemezse 0 olarak döndür
    return gpu_usage

# CPU ve Bellek Kullanımını Ölçmek İçin Fonksiyon
def get_cpu_memory_usage():
    cpu_usage = psutil.cpu_percent(interval=1)  # CPU yüzdesi
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)  # GB cinsinden bellek kullanımı
    return cpu_usage, memory_usage

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Eğitim ve Test Setlerine Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Özellikleri Ölçeklendirme
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA ve LDA Dönüşümleri
pca = PCA(n_components=10)
lda = LDA()

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

# Algoritmalar ve İsimleri
models = {
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=200),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "SVM": SVC(probability=True, random_state=42),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000)
}

# Sonuçları Saklamak İçin Liste
resource_usage_results = []

# PCA, LDA ve No Reduction Durumlarını İçeren Modellerin CPU, GPU ve Bellek Kullanımının Hesaplanması
reductions = {
    "No Reduction": (X_train_scaled, X_test_scaled),
    "With PCA": (X_train_pca, X_test_pca),
    "With LDA": (X_train_lda, X_test_lda)
}

# Normal Eğitim ve Test Süreleri
for reduction_name, (X_tr, X_te) in reductions.items():
    for model_name, model in models.items():
        # CPU ve Bellek Kullanımı Ölçme
        cpu_usage, memory_usage = get_cpu_memory_usage()

        # Modeli Eğitme
        model.fit(X_tr, y_train)

        # GPU Kullanımı Ölçme
        gpu_usage = get_gpu_usage()

        # Sonuçları Listeye Ekleyin
        resource_usage_results.append({
            "Reduction": reduction_name,
            "Model": model_name,
            "CPU Usage (%)": cpu_usage,  # CPU Yüzdesi
            "GPU Usage (%)": gpu_usage,  # GPU Yüzdesi
            "Memory Usage (GB)": memory_usage  # Anlık Bellek Kullanımı
        })

# Sonuçları DataFrame Olarak Gösterme
resource_usage_df = pd.DataFrame(resource_usage_results)

# Performans Sonuçları Tablosunu Görüntüleme
print("GPU/CPU ve Bellek Kullanımı Tablosu:")
print(resource_usage_df)

C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:29:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:34:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\hsynt\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [00:38:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GPU/CPU ve Bellek Kullanımı Tablosu:
       Reduction                Model  CPU Usage (%)  GPU Usage (%)  \
0   No Reduction        Random Forest            5.0            0.0   
1   No Reduction              XGBoost            2.0            0.0   
2   No Reduction                  SVM            7.9            0.0   
3   No Reduction                 k-NN            2.2            0.0   
4   No Reduction  Logistic Regression            2.4            0.0   
5       With PCA        Random Forest            2.1            0.0   
6       With PCA              XGBoost            0.7            0.0   
7       With PCA                  SVM            6.2            0.0   
8       With PCA                 k-NN            0.5            0.0   
9       With PCA  Logistic Regression            0.5            0.0   
10      With LDA        Random Forest            1.4            0.0   
11      With LDA              XGBoost            0.6            0.0   
12      With LDA                  SVM   